<font color=gray>Oracle Cloud Infrastructure Data Science Demo Notebook

Copyright (c) 2021 Oracle, Inc.<br>
Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>

# Invoke your model that has been deployed through Data Science Model Deployment

**NOTE: Double-check that this notebook is running in the `tensorflow26_p37_cpu_v1` conda kernel.**


In [ ]:
import os
import requests
import oci
from oci.signer import Signer

from ads.model.deployment import ModelDeployer, ModelDeploymentProperties

## Invoke the Model HTTP Endpoint 

Once a model is successfully deployed, the next step is to invoke the model `/predict` endpoint. 

In the cell below, we define a JSON payload that we pass to the model endpoint. This payload corresponds to the first five rows of the training dataset defined in notebook `1-model-training.ipynb`. 

In [ ]:
input_data='{"Age":{"0":42,"1":50,"2":38,"3":34,"5":33},"TravelForWork":{"0":"infrequent","1":"often","2":"infrequent","3":"often","5":"often"},"SalaryLevel":{"0":5054,"1":1278,"2":6296,"3":6384,"5":4608},"JobFunction":{"0":"Product Management","1":"Software Developer","2":"Software Developer","3":"Software Developer","5":"Software Developer"},"CommuteLength":{"0":2,"1":9,"2":3,"3":4,"5":3},"EducationalLevel":{"0":"L2","1":"L1","2":"L2","3":"L4","5":"L2"},"EducationField":{"0":"Life Sciences","1":"Life Sciences","2":"Other","3":"Life Sciences","5":"Life Sciences"},"EnvironmentSatisfaction":{"0":2,"1":3,"2":4,"3":4,"5":4},"Gender":{"0":"Female","1":"Male","2":"Male","3":"Female","5":"Male"},"HourlyRate":{"0":94,"1":61,"2":92,"3":56,"5":79},"JobInvolvement":{"0":3,"1":2,"2":2,"3":3,"5":3},"JobRole":{"0":"Sales Executive","1":"Research Scientist","2":"Laboratory Technician","3":"Research Scientist","5":"Laboratory Technician"},"JobSatisfaction":{"0":4,"1":2,"2":3,"3":3,"5":4},"MaritalStatus":{"0":"Single","1":"Married","2":"Single","3":"Married","5":"Single"},"MonthlyRate":{"0":19479,"1":24907,"2":2396,"3":23159,"5":11864},"NumCompaniesWorked":{"0":8,"1":1,"2":6,"3":1,"5":0},"OverTime":{"0":"Yes","1":"No","2":"Yes","3":"Yes","5":"No"},"PercentSalaryHike":{"0":11,"1":23,"2":15,"3":11,"5":13},"PerformanceRating":{"0":3,"1":4,"2":3,"3":3,"5":3},"RelationshipSatisfaction":{"0":1,"1":4,"2":2,"3":3,"5":3},"StockOptionLevel":{"0":0,"1":1,"2":0,"3":0,"5":0},"YearsinIndustry":{"0":8,"1":10,"2":7,"3":8,"5":8},"TrainingTimesLastYear":{"0":0,"1":3,"2":3,"3":3,"5":2},"WorkLifeBalance":{"0":1,"1":3,"2":3,"3":3,"5":2},"YearsAtCurrentLevel":{"0":4,"1":7,"2":0,"3":7,"5":7},"YearsSinceLastPromotion":{"0":0,"1":1,"2":0,"3":3,"5":3},"name":{"0":"Tracy Moore","1":"Andrew Hoover","2":"Julie Bell","3":"Thomas Adams","5":"Rhonda Grant"}}'

Next, **paste the model deployment uri in the cell below**. You can find the uri in the OCI console in the details page of your model deployment under "Resources > Invoking Your Model".  

This cell also lets you configure the method of authentication you want to use with your model endpoint. Two options are available: 
* user principal (aka config + key approach) 
* resource principal 

The cell defaults to resource principal. If you prefer to use user principal, set the flag `usign_rps` to `False`. 

In [ ]:
# Using resource principals. You can alternatively use the config+key flow. 
using_rps = True
# Replace with the uri of your model deployment: 
uri = ''

if using_rps: # using resource principal:     
    auth = oci.auth.signers.get_resource_principals_signer()
else: # using config + key: 
    config = oci.config.from_file("~/.oci/config") # replace with the location of your oci config file
    auth = Signer(
        tenancy=config['tenancy'],
        user=config['user'],
        fingerprint=config['fingerprint'],
        private_key_file_location=config['key_file'],
        pass_phrase=config['pass_phrase'])

Finally we submit a request to the model endpoint using the payload defined in `input_data`. 

The model trained in notebook `1-model-training.ipynb` should return values `[True, False, True, False, False]`. 

In [ ]:
%%time 

# submit request to model endpoint: 
requests.post(uri, json=input_data, auth=auth).json()